# LAB4b.2 - Disambiguating with existing tools

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

There are many existing entity linking tools out there. We will not build our own, but instead run two existing tools and dig deeper into their output. In order to do this, we will do the following steps:
1. Setup your environment (installation of modules and download of dataset)
2. Load the dataset
3. Run disambiguation with our tools on top of recognized mentions
4. Run entity annotation from scratch: recognition and disambiguation together

### 1. Setup your environment

#### 1.1 Install the needed modules

For the purpose of this week's coding exercises, we will need some new libraries that are probably not installed on your computer:
* [rdflib](https://pypi.org/project/rdflib/)
* (perhaps) [lxml](https://pypi.org/project/lxml/)

You should install these libraries using `conda` or `pip`.

Let's now check if all needed libraries are installed on your computer and can be imported. 

In [1]:
from rdflib import Graph, URIRef
from tqdm import tqdm ## to create progress bar to measure progress
import sys
import requests
import urllib
import urllib.parse
from urllib.request import urlopen, Request
from urllib.parse import urlencode
import xml.etree.cElementTree as ET
from lxml import etree
import time
import json

# import our own utility functions and classes
import lab4_utils as utils
import lab4_classes as classes

**If you see any errors with the imports:**
* read the error carefully
* install the library that is missing
* try to execute the imports again

Don't proceed before the imports work; we will use these modules in the explanation below and in the assignment.

#### 1.2 Obtain the N3 Reuters-128 dataset

**This step should be done if you downloaded or cloned the course repository from GitHub.**

We will work with a small dataset called Reuters-128. This dataset contains 128 Reuters documents annotated with entity mentions and links to DBpedia. You probably have this dataset if you downloaded or cloned the course repository from GitHub. Otherwise, you can download it from https://raw.githubusercontent.com/dice-group/n3-collection/master/Reuters-128.ttl.

Store the dataset file 'Reuters-128.ttl' in the same directory as this notebook.

*Before proceding, please verify that your setup of libraries (1.1) is correct and that the dataset is downloaded in the right location (1.2).*

Congratulations, you are all set up!

### 2. Load the data from N3

**Let's now parse this dataset to a list of news item objects that contain: 1) the text and 2) the entity mentions+links for each news item.**

The data is in a .ttl format called [NIF](https://nif.readthedocs.io/en/latest/) (don't worry about these formats for now, we provide a function to parse this dataset to python classes).
You can open the .ttl file in Jupyter or a text editor for inspection. You will see records as the following, where the first represents a Reuters text and the second record represents an entity ''Dillon Read and Co'' that is mentioned in the text:

```
<http://aksw.org/N3/Reuters-128/41#char=0,481>
      a       nif:String , nif:Context , nif:RFC5147String ;
      nif:beginIndex "0"^^xsd:nonNegativeInteger ;
      nif:endIndex "481"^^xsd:nonNegativeInteger ;
      nif:isString "Avnet Inc said it filed with the Securities and Exchange Commission a registration statement for a proposed public offering of 150 mln dlrs of convertible subordinated debentures due 2012. Avnet said it will use the net proceeds for general working capital purposes and the anticipated domestic and foreign expansion of its distribution, assembly and manufacturing businesses. The company said an investment banking group managed by Dillon Read and Co Inc will handle the offering."@en ;
      nif:sourceUrl <http://www.research.att.com/~lewis/Reuters-21578/15054> .
      
<http://aksw.org/N3/Reuters-128/41#char=433,455>
      a       nif:RFC5147String ;
      nif:anchorOf "Dillon Read and Co Inc"^^xsd:string ;
      nif:beginIndex "433"^^xsd:nonNegativeInteger ;
      nif:endIndex "455"^^xsd:nonNegativeInteger ;
      nif:referenceContext
              <http://aksw.org/N3/Reuters-128/41#char=0,481> ;
      itsrdf:taIdentRef <http://dbpedia.org/resource/Dillon,_Read_%26_Co.> ;
      itsrdf:taSource "DBpedia_en_3.9"^^xsd:string .
```

We provided the python code in *lab4_utils.py* and *lab4_classes.py* to process these NIF structures and to get the data. Make sure these files are in the same location as the notebook that you are running. If not, you should have seen an error with the import function above. If succesfully imported, you can now use the functions that are defined in these files.

In [2]:
# Relative path to the data file if in the same location as the notebook
reuters_file='Reuters-128.ttl'

The *load_article_from_nif_file* function accesses the files and creates a set of data object of the type `NewsItem` containing among others the text, the mentions of entities and their URI's to DBPedia.

In [3]:
articles=utils.load_article_from_nif_file(reuters_file)

**Data description**

If you check the *lab4_utils.py* file, you will see that `articles` is a list of news articles (members of the `NewsItem` which we defined in the file *lab4_classes.py*). To help you understand, we now present the information that we store for each news article. 

Each news item contains the following fields: `identifier` (the original identifier of this document in the dataset), `dct` (the document creation time, if known), `content` (the text of this document), and `entity_mentions` (a list of entity mention occurrences that were found in this document).

For each of the entity mentions, we store the following information: `sentence` (in which sentence was this mention found), `mention` (the exact phrase of this mention), `the_type` (its type, if known), `begin_index` (the starting offset of this mention), `end_index` (the ending offset of this mention), `gold_link` (the gold link for this mention), `aida_link` (the link for this mention proposed by AIDA), and `spotlight_link` (the link for this mention proposed by Spotlight).

In [4]:
print(len(articles))

128


In [5]:
article=articles[0]
type(article)

lab4_classes.NewsItem

### 3. Disambiguation with existing entity linking tools

We will perform entity linking on this dataset with two modern tools, called AIDA and DBpedia Spotlight.

#### 3.1 AIDA

AIDA is an entity linking system that puts all entity candidates in a graph network and then performs a probabilistic optimization to find the best connected candidate in this graph for each of the entity mentions. We call this **collective** or **global** disambiguation, because all entity mentions are disambiguated together.

AIDA uses two types of connections in their algorithm: 
* The first type is between a mention and an entity instance and tells us how often is an instance is referred to by a mention. For example, the mention "Jordan" refers most often to the country, while less often to the basketball player, Michael Jordan. 
* The second connection type is between two entity instances; it tells us how well-connected are two entities. For example, the country Jordan is better connected with Saudi Arabia then it is with the basketball club San Antonio.

You can play with AIDA by using their demo page: https://gate.d5.mpi-inf.mpg.de/webaida/.

Here is a text you can try:
```
The President opened a window into the state of his mind Sunday when he lashed out against Jennifer Williams, an aide to Vice President Mike Pence, who described his July 25 call with Ukrainian President Volodymyr Zelensky in her deposition as "inappropriate."
```

More description can be found in their paper: http://www.aclweb.org/anthology/D11-1072.

In the function `aida_disambiguation` below, we do the following: 
* we iterate the 128 documents of the Reuters-128 dataset (called `articles`)
* for each document, we combine the text and the marking of the entity mentions in a single string `new_content`
* we send a request to AIDA to disambiguate the entities in this string
* we store the entity links back to the `articles` in the field `aida_link`

**Note:** You might encounter an SSL error such as `SSL: CERTIFICATE_VERIFY_FAILED` when running the urlopen command. This happens sometimes when your browser does not trust the certificate of some pages, it is a safety mechanism. Do the following to tell your browser that it can trust the web service we use:

```import ssl
context = ssl._create_unverified_context()
```
Then use this context in the urlopen call:

`urllib.urlopen("https://your-url", context=context)`


AIDA returns a json data structure which looks as follows:

```
{"originalFileName":null,"originalText":"Western Digtial Corp said it has completed the redemption of its 47 mln dlr, 6-3\/4 pct convertible subordinated debentures due 2011. The redemption resulted in the issuance of 2,685,142 shares of the companys common stock, Western Digital said.","overallTime":"16","allEntities":["YAGO:Western_Digital"],"entityMetadata":{"AIDA:--OOKBE--":{},"YAGO:Western_Digital":{"knowledgebase":"YAGO","depictionurl":"http:\/\/upload.wikimedia.org\/wikipedia\/en\/f\/f7\/WesternDigitalLogo.svg","depictionthumbnailurl":"http:\/\/upload.wikimedia.org\/wikipedia\/en\/thumbWesternDigitalLogo.svg\/200px-WesternDigitalLogo.svg","importance":0.01596538400793088,"entityId":"Western_Digital","type":["YAGO_yagoPermanentlyLocatedEntity","YAGO_yagoLegalActorGeo","YAGO_wordnet_organization_108008335","YAGO_wordnet_language_unit_106284225","YAGO_wordnet_name_106333653","YAGO_wikicategory_Companies_based_in_Irvine\u002c_California","YAGO_wordnet_relation_100031921","YAGO_wikicategory_Computer_storage_companies","YAGO_wikicategory_American_brands","YAGO_yagoLegalActor","YAGO_wordnet_abstraction_100002137","YAGO_wordnet_company_108058098","YAGO_wikicategory_Companies_established_in_1970","YAGO_wordnet_social_group_107950920","YAGO_wordnet_institution_108053576","YAGO_wordnet_entity_100001740","YAGO_wordnet_part_113809207","YAGO_wordnet_group_100031264","YAGO_wordnet_trade_name_106845599","YAGO_wikicategory_Companies_listed_on_NASDAQ"],"readableRepr":"Western Digital","url":"http:\/\/en.wikipedia.org\/wiki\/Western%20Digital"}},"annotatedText":"[[AIDA:--OOKBE--|Western Digtial Corp]] said it has completed the redemption of its 47 mln dlr, 6-3\/4 pct convertible subordinated debentures due 2011. The redemption resulted in the issuance of 2,685,142 shares of the companys common stock, [[YAGO:Western_Digital|Western Digital]] said.","docID":"-1544055862_1579203938530","mentions":[{"allEntities":[],"offset":0,"name":"Western Digtial Corp","length":20},{"allEntities":[{"kbIdentifier":"YAGO:Western_Digital","disambiguationScore":"1"}],"offset":223,"name":"Western Digital","length":15,"bestEntity":{"kbIdentifier":"YAGO:Western_Digital","disambiguationScore":"1"}}],"formatVersion":"2.3","allTypes":["YAGO_wordnet_trade_name_106845599","YAGO_wikicategory_Companies_listed_on_NASDAQ","YAGO_wordnet_name_106333653","YAGO_wordnet_organization_108008335","YAGO_wikicategory_Computer_storage_companies","YAGO_yagoLegalActorGeo","YAGO_wordnet_institution_108053576","YAGO_wikicategory_American_brands","YAGO_wikicategory_Companies_established_in_1970","YAGO_wordnet_relation_100031921","YAGO_wordnet_abstraction_100002137","YAGO_yagoPermanentlyLocatedEntity","YAGO_wordnet_social_group_107950920","YAGO_wordnet_entity_100001740","YAGO_wordnet_language_unit_106284225","YAGO_yagoLegalActor","YAGO_wikicategory_Companies_based_in_Irvine\u002c_California","YAGO_wordnet_company_108058098","YAGO_wordnet_part_113809207","YAGO_wordnet_group_100031264"]
```

The next function extracts the information from each article object to query AIDA and processes the json structure to obtain the best scoring links. These links are stored as results for the mentions in the article structure.

In [4]:
def aida_disambiguation(articles, aida_url):
    """
    Perform disambiguation with AIDA.
    """
    with tqdm(total=len(articles), file=sys.stdout) as pbar:  #pbar provides a nice progress bar for the interation over the articles
        for i, article in enumerate(articles):
                                    
            # AIDA expects entity mentions that are pre-marked inside text. 
            # For example, the sentence "Obama visited Paris today", 
            # should be transformed to "[[Obama]] visited [[Paris]] today."
            # We do this in the next 5 lines of code.
            original_content = article.content
            new_content=original_content
            for entity in reversed(article.entity_mentions):
                entity_span=new_content[entity.begin_index: entity.end_index]
                new_content=new_content[:entity.begin_index] + '[[' + entity_span + ']]' + new_content[entity.end_index:]

            # Now, we can run the AIDA library with this string.
            params={"text": new_content, "tag_mode": 'manual'}
            request = Request(aida_url, urlencode(params).encode())
            # AIDA returns a json structure
            this_json = urlopen(request).read().decode('unicode-escape')
            try:
                results=json.loads(this_json)
            except:
                continue
            # print(this_json)
            # Let's normalize the disambiguated entities.
            # This means mostly removing the first part of the URI which is always the same (YAGO:)
            # and leaving only the entity identification part (e.g., Barack_Obama).
            dis_entities={}
            # We iterate over the data elements "mentions" in the json results
            for dis_entity in results['mentions']:
               # print(dis_entity)
                ## AIDI labels the bestEntity in the json
                if 'bestEntity' in dis_entity.keys():
                    best_entity=dis_entity['bestEntity']['kbIdentifier']
                    clean_url=best_entity[5:] #SKIP YAGO:
                else:
                    clean_url='NIL'
                dis_entities[str(dis_entity['offset'])] = clean_url # BECOMES THE VALUE IN THE DICTIONARY FOR THE OFFSET(REPRESENTING THE START OF THE MENTION) IN THE TEXT
                
            # We can now store the entity to our class instance for later processing.
            for entity in article.entity_mentions:
                start = entity.begin_index
                try:
                    dis_url = str(dis_entities[str(start)])  # WE GET THE DISAMBIGUATED URL
                except:
                    dis_url='NIL'
                entity.aida_link = dis_url  # THE ENTITY IS ENRICHED WITH THE AIDA_LINK

            # The next two lines only update the progress bar
            pbar.set_description('processed: %d' % (1 + i))
            pbar.update(1)
    return articles

In [5]:
# AIDA is running in an external location - for this reason, we need to send an HTTP request. This should take a few minutes.
aida_disambiguation_url = "https://gate.d5.mpi-inf.mpg.de/aida/service/disambiguate"
### We define a subset to test the function
test_items=articles[0:5]
processed_aida=aida_disambiguation(test_items, aida_disambiguation_url)

  0%|          | 0/5 [01:21<?, ?it/s]


URLError: <urlopen error [Errno 60] Operation timed out>

In [10]:
#### Now we are ready to do the full set:
processed_aida=aida_disambiguation(articles, aida_disambiguation_url)

processed: 2:   2%|▏         | 2/128 [00:00<00:24,  5.18it/s]

/Users/piek/.local/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\/'


processed: 15:  12%|█▏        | 15/128 [00:24<03:00,  1.60s/it]


KeyboardInterrupt: 

**Note:** The progress bar is sometimes stopping before 128. To be sure whether the command is done executing, please pay attention to the asterisk sign left from the cell. Once the '\*' sign turns into a number, you can be sure that you can proceed with the notebook! Also, look at the first printed number (left from the progress bar), it should say “processed: 128” when it is done.

In [8]:
print(len(processed_aida))

5


The next loop print all the links for each mention.

In [18]:
# print the URLs for the entities in the articles, we break after the first
for article in processed_aida:
    for entity in article.entity_mentions:
        print(entity.aida_link)
    

NIL
Western_Digital
NIL
NIL
NIL
New_York_Stock_Exchange
Philips
NIL
Netherlands
NASDAQ
New_York_Stock_Exchange
Philips
New_York
United_States
Philips
NIL
NIL
IBM
Content_management_system
IBM
NIL
NIL
NIL
First_Boston
NIL
NIL


#### 3.2 DBpedia Spotlight

DBpedia Spotlight (or only "Spotlight" for brevity) is an entity recognition and linking tool that performs linking to DBpedia. The core of their method is a vector space model to compute similarity between the text to annotate and the Wikipedia pages of all entity candidates for a mention. Then the entities with largest similarity are chosen.

You can try out their demo as well: http://dbpedia-spotlight.github.com/demo/.

Here is their paper: http://oa.upm.es/8923/1/DBpedia_Spotlight.pdf

In the function `spotlight_disambiguate` below, we do the following: 
* we iterate the 128 documents of the Reuters-128 dataset (called `articles`)
* for each document, we create the input spotlight expects - a json with the content of the article and a confidence threshold
* we send a request to Spotlight to disambiguate the entities in this string
* we store the entity links back to the `articles` in the field `spotlight_link`

In [64]:
def spotlight_disambiguate(articles, spotlight_url):
    """
    Perform disambiguation with DBpedia Spotlight.
    """
    with tqdm(total=len(articles), file=sys.stdout) as pbar:
        for i, article in enumerate(articles):
            # Similar as with AIDA, we first prepare the document text and the mentions
            # in order to provide these to Spotlight as input.
            
            # Parameters 
            # 'text' - text to be annotated 
            # 'confidence' -   confidence score for linking
            confidence = 0.35
            params = {"text" : article.content , "confidence" : confidence}
            print(params)
            # Response content type
            headers = {'accept': 'application/json'}

            # GET Request
            results = requests.get(spotlight_url, params=params, headers=headers)
            if results.status_code != 200:
                # Something went wrong
                raise APIError(results.status_code)
            
            # Note that you can adjust the confidence value. Check the online demo to see the effect. 
            # What will happen with the recall and precision if you increase the confidence?
            
            # Process the results and normalize the entity URIs
            print('results:', results)
            
            if not results:
                continue
            
            j=results.json()
            dis_entities={}
            if 'Resources' in j: 
                resources=j['Resources']
            else: 
                resources=[]
            for dis_entity in resources:
                dis_entities[str(dis_entity['@offset'])] = utils.normalizeURL(dis_entity['@URI'])
            
            # Let's now store the URLs by Spotlight to our class for later analysis.
            for entity in article.entity_mentions:
                start = entity.begin_index
                if str(start) in dis_entities:
                    dis_url = dis_entities[str(start)]
                else:
                    dis_url = 'NIL'
                entity.spotlight_link = dis_url
    
            # The next two lines only update the progress bar
            pbar.set_description('processed: %d' % (1 + i))
            pbar.update(1)
                
            # Pause for 100ms to prevent overloading the server
            time.sleep(0.1)
    return articles

In [65]:
# Spotlight is running in an external location - for this reason, we need to send an HTTP request. Hopefully, this will not take a long time.
# https://www.dbpedia-spotlight.org/api

# New API:
spotlight_disambiguation_url="https://api.dbpedia-spotlight.org/en/annotate"
### We add the dbspotlight links to same processed_data

test_items_processed_by_spotligh=spotlight_disambiguate(test_items, spotlight_disambiguation_url)

  0%|          | 0/5 [00:00<?, ?it/s]{'text': rdflib.term.Literal('Best Buy Co Inc said its sales for March rose to 26.1 mln dlrs from 11.9 mln dlrs in the comparable 1986 period. It said sales for the fiscal year ended March 31 rose to 329.5 mln dlrs from 113.1 mln dlrs a year earlier. The company said sales performance was based on the addition of 12 new retail facilities.', lang='en'), 'confidence': 0.35}
results: <Response [200]>
processed: 1:  20%|██        | 1/5 [00:00<00:00,  5.82it/s]{'text': rdflib.term.Literal('Great Lakes Chemical Corp said it registered with the Securities and Exchange Commission a proposed public offering of two mln shares of its common stock. Proceeds from the offering will be used to repay debt incurred in the acquisition of two chemical companies, to increase Great Lakes equity investment in Huntsman Chemical Corp and for general corporate purposes, it said. Underwriters are led by First Boston Corp, Goldman, Sachs and Co and Eberstadt Fleming Inc.', la

In [67]:
# print the URLs for the entities in the articles, we break after the first
for article in test_items_processed_by_spotligh:
    for entity in article.entity_mentions:
        print(entity.spotlight_link)


Best_Buy
Great_Lakes_Chemical_Corporation
NIL
Huntsman_Corporation
First_Boston
Goldman_Sachs
Eberstadt
NIL
Saint_Joseph
NIL
New_Jersey
Pasha
Erskine
Pasha
Erskine


If it works and you also managed to get AIDA working, we can now run Spotligh on the AIDA output

In [ ]:
processed_both=spotlight_disambiguate(processed_aida, spotlight_disambiguation_url)

In [20]:
print(len(processed_both))

5


#### 3.3 Comparing the output of AIDA and Spotlight to the gold links

Because we stored the decisions by both tools in our list `articles`, we can now compare their output.

Let's pick an article and print the decisions made by AIDA and Spotlight on that article, and then compare that to the gold link.

In [21]:
article_number=6

an_article=processed_both[article_number]
doc_id=an_article.identifier
print(doc_id)
for m in an_article.entity_mentions:
    print('|mention: %s\t|gold:\t%s\t|aida:\t%s\t|spotlight:\t%s |' % (m.mention, m.gold_link, m.aida_link, m.spotlight_link))

IndexError: list index out of range

## End of this notebook